# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He; psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770920312102                   -0.52    9.0         
  2   -2.772150183437       -2.91       -1.32    1.0    104ms
  3   -2.772170286328       -4.70       -2.47    1.0    139ms
  4   -2.772170656367       -6.43       -3.20    1.0    107ms
  5   -2.772170722791       -7.18       -4.27    2.0    124ms
  6   -2.772170722867      -10.12       -4.39    1.0    140ms
  7   -2.772170723013       -9.84       -5.33    1.0    113ms
  8   -2.772170723014      -11.81       -5.49    2.0    128ms
  9   -2.772170723015      -12.07       -6.19    1.0    125ms
 10   -2.772170723015      -12.98       -6.57    1.0    126ms
 11   -2.772170723015   +  -13.75       -6.91    1.0    128ms
 12   -2.772170723015      -13.75       -7.95    1.0    129ms
 13   -2.772170723015   +  -14.35       -7.87    2.0    149ms
 14   -2.772170723015   +  -14.40       -8.36    1.0    120ms
n     Ene

1.7735580312637835

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770735590979                   -0.53    9.0         
  2   -2.772049320186       -2.88       -1.31    1.0    104ms
  3   -2.772082507388       -4.48       -2.59    1.0    105ms
  4   -2.772083411625       -6.04       -3.78    2.0    162ms
  5   -2.772083417014       -8.27       -4.08    2.0    123ms
  6   -2.772083417789       -9.11       -5.23    1.0    111ms
  7   -2.772083417810      -10.67       -5.70    2.0    143ms
  8   -2.772083417811      -12.49       -6.29    1.0    114ms
  9   -2.772083417811      -13.09       -7.28    2.0    148ms
 10   -2.772083417811   +  -14.88       -8.09    2.0    144ms

Polarizability via ForwardDiff:       1.7725349480420538
Polarizability via finite difference: 1.7735580312637835
